## 패키지 설치

In [7]:
!pip install -U financedatabase
!pip install -U yahoofinancials

## env 설정

In [2]:
import financedatabase as fd
from yahoofinancials import YahooFinancials
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import os

## ETF 리스트 추출 wtih fd

In [3]:
etfs = fd.ETFs()
symbols = list(etfs.select().index)

## ETF 데이터베이스 확인 (Yahoo)

In [69]:
today_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365 * 20)).strftime('%Y-%m-%d')
print(f'{start_date}-{today_date}')
business_days = pd.date_range(start=start_date, end=today_date, freq='B')

2004-01-02-2023-12-28


In [71]:
folder_name = 'data'
folder_path = os.path.join(os.getcwd(), folder_name)
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

In [1]:
error_symbol = []

for symbol in tqdm(symbols):
  try:
    yahoo_financials = YahooFinancials(symbol)
    prices = yahoo_financials.get_historical_price_data(start_date, today_date, 'daily')
    prices = pd.DataFrame(prices.get(symbol).get('prices')).drop(['date'],axis=1)
    prices = prices.rename(columns={'high':'High', 'low':'Low', 'open':'Open', 'close':'Close', 'volume':'Volume', 'adjclose':'Adj Close','formatted_date':'Date'})
    prices = prices[['Date','Open','High','Low','Close','Adj Close','Volume']]
    prices = prices.set_index("Date")
    prices = prices.set_index(pd.to_datetime(prices.index)).asfreq('B')
    prices = prices.ffill().reset_index()
    prices.to_csv(f"./{folder_name}/{symbol}.csv")
  except:
    error_symbol.append(symbol)

#save error symbols
df_err = pd.DataFrame({"Symbol": error_symbol})
csv_file_path = "error_symbols.csv"
df_err.to_csv(csv_file_path, index=False)

## Save ETF data

In [ ]:
from google.colab import files
!zip -r /content/data.zip /content/data
files.download('/content/data.zip')
files.download('/content/error_symbols.csv')

---